In [1]:
import pandas as pd
import pytz
import numpy as np
import os
from sklearn import preprocessing
import re
import matplotlib
from matplotlib.patches import Polygon, Rectangle
matplotlib.use('Qt5Agg')
from datetime import timedelta
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from casadi import *
import calendar
import casadi as cd
from sklearn.linear_model import LinearRegression

Reading the bill data and the power data

In [2]:
def custom_date_parser(date_string):
    return pd.to_datetime(date_string, format='%d-%m-%Y %H:%M:%S')

# Specify the path to the main directory containing folders and files
path = 'D:\\mlinternship\\iitgdata'
folders = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]
df_list = []

# Iterate through each folder
for folder in folders:
    # Construct the full path to the current folder
    folder_path = os.path.join(path, folder)
    # Iterate through files in the current folder
    for filename in os.listdir(folder_path):
        # Check if the file has the '.xlsx' extension
        if filename.endswith('.xlsx'):
            # Construct the full path to the Excel file
            file_path = os.path.join(folder_path, filename)

            # Use the custom date parser function
            df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)

            # Append the dataframe to the list
            df_list.append(df)


bill_path = 'D:\\mlinternship\\IITGuwahatiElectricityBills'
for filename in os.listdir(bill_path):
    # Check if the file has the '.xlsx' extension
    if filename.endswith('.xlsx'):
        file_path = os.path.join(bill_path, filename)
        # Use the custom date parser function
        bill_df = pd.read_excel(file_path)

bill_df['Month'] = pd.to_datetime(bill_df['Month'])
#I assumed its in kilowatts
bill_df['MW'] = bill_df['Number of units of electricity consumed']/1000
bill_df.drop(['Number of units of electricity consumed'], axis=1)
bill_df

C:\Users\jaina\AppData\Local\Temp\ipykernel_33656\1131752821.py:21: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)
C:\Users\jaina\AppData\Local\Temp\ipykernel_33656\1131752821.py:21: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)
C:\Users\jaina\AppData\Local\Temp\ipykernel_33656\1131752821.py:21: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_excel(file_path, header=3, date_pars

,Month,MW
0,2022-01-01,1415.4245
1,2022-02-01,1353.4260
2,2022-03-01,2254.4145
3,2022-04-01,2269.1010
4,2022-05-01,2690.4150
5,2022-06-01,2708.4910
6,2022-07-01,3113.6625
7,2022-08-01,3606.0495
8,2022-09-01,3335.3190
9,2022-10-01,2787.5835


,Month,Number of units of electricity consumed,MW
0,2022-01-01,1415424.5,1415.4245
1,2022-02-01,1353426.0,1353.4260
2,2022-03-01,2254414.5,2254.4145
3,2022-04-01,2269101.0,2269.1010
4,2022-05-01,2690415.0,2690.4150
5,2022-06-01,2708491.0,2708.4910
6,2022-07-01,3113662.5,3113.6625
7,2022-08-01,3606049.5,3606.0495
8,2022-09-01,3335319.0,3335.3190
9,2022-10-01,2787583.5,2787.5835


Reading the temperature data and merging the power and temperature data

In [3]:
#read the power data
power_df = pd.concat(df_list, ignore_index=True)
power_df.to_csv('power_datacsv.csv')
power_df['Time'] = pd.to_datetime(power_df['Time'])
power_df['Time'] = power_df['Time'].round('min')
#replace all the 'NR' values in MW column to NaN
power_df['MW'] = power_df['MW'].replace('NR', np.nan)
power_df['MW'] = power_df['MW'].replace('nr', np.nan)
power_df = power_df[['Time', 'MW']]
power_df['MW'] = power_df['MW'].astype(str)
power_df['MW'] = pd.to_numeric(power_df['MW'].str.replace(',', '.'), errors='coerce')
power_df['Time'] = pd.to_datetime(power_df['Time'])
power_df = power_df.sort_values('Time')
full_power_df = power_df.copy()

# read the temperature data csv
temperature_data_csv_path = 'D:\\mlinternship\\iitgdata\\temperaturedata'
filename = 'guwahati_temperature_data.csv'
file = os.path.join(temperature_data_csv_path, filename)
temperature_df = pd.read_csv(file)
temperature_df.rename(columns={'valid': 'Time'}, inplace = True)
temperature_df = temperature_df.rename(columns={'tmpc': 'temperature'})
temperature_df = temperature_df[['Time', 'temperature']]
temperature_df['Time'] = pd.to_datetime(temperature_df['Time'])
temperature_df['Time'] = pd.DatetimeIndex(temperature_df['Time']) + timedelta(hours=5,minutes=30)
temperature_df['temperature'] = pd.to_numeric(temperature_df['temperature'], errors='coerce')
temperature_df.set_index('Time', inplace=True)
temperature_df['temperature'] = temperature_df['temperature'].interpolate(method='polynomial', order = 5)
temperature_df.reset_index(inplace=True)

# joining the two dataframes such that the temperature data is only taken if there exists a reading in the power data dataframe
df = pd.merge(power_df, temperature_df, on='Time', how='left')
df['temperature'] = df['temperature'].interpolate(method='polynomial', order = 5)
df['Time'] = pd.to_datetime(df['Time'])

full_model_start_time = pd.Timestamp('2022-02-07 00:00:00')
full_model_end_time = pd.Timestamp('2022-12-25 23:00:00')
df = df[(df['Time'] >= full_model_start_time) & (df['Time'] <= full_model_end_time)]
df = df.drop(df[df['MW'] > 20].index)
df = df.sort_values('Time')
df.reset_index(drop=True)
df

,Time,MW,temperature
0,2022-02-07 00:00:00,2.18,12.000000
1,2022-02-07 01:00:00,2.05,11.633781
2,2022-02-07 02:00:00,NaN,11.703556
3,2022-02-07 03:00:00,1.71,12.000000
4,2022-02-07 04:00:00,NaN,12.000000
...,...,...,...
5753,2022-12-25 18:00:00,NaN,19.000000
5754,2022-12-25 19:00:00,2.60,18.000000
5755,2022-12-25 20:00:00,NaN,18.000000
5756,2022-12-25 21:00:00,2.60,18.000000


,Time,MW,temperature
160,2022-02-07 00:00:00,2.18,12.000000
161,2022-02-07 01:00:00,2.05,11.633781
162,2022-02-07 02:00:00,NaN,11.703556
163,2022-02-07 03:00:00,1.71,12.000000
164,2022-02-07 04:00:00,NaN,12.000000
...,...,...,...
5914,2022-12-25 18:00:00,NaN,19.000000
5915,2022-12-25 19:00:00,2.60,18.000000
5916,2022-12-25 20:00:00,NaN,18.000000
5917,2022-12-25 21:00:00,2.60,18.000000


Creating the CDH and Omega columns

In [4]:
TcoolStPt = 31
CDH = df['temperature'] - TcoolStPt
CDH.clip(lower=0, inplace=True)
CDH = pd.DataFrame(data=CDH.values, columns=['CDH'], index=df.index)
# Concatenate CDH with the original DataFrame using the index
df = pd.concat([df, CDH], axis=1)
df = df.sort_values('Time')
df.reset_index(inplace=True, drop = True)

numOmegas = 24 * 7
num_of_rows = df.shape[0]
omegas = np.zeros((num_of_rows, numOmegas))  # Assuming numOmegas columns for omegas
concatenated_data = np.concatenate((df, omegas), axis=1)
column_names = ['Time', 'MW', 'temperature', 'CDH']
for i in range(1, numOmegas + 1,1):
    column_names.append('omega' + str(i))

df = pd.DataFrame(concatenated_data, columns=column_names)
df['Time'] = pd.to_datetime(df['Time'])
for i in range(0,num_of_rows):
        datetime = df.Time.loc[i]
        hourOfWeekIndex = int(datetime.dayofweek*24+(datetime.hour+1))
        x = np.zeros((1,numOmegas))
        x[0,hourOfWeekIndex-1]=1
        omegas[i,:]=x

df.iloc[:,4:]=omegas
DF = df.copy()
df = df.dropna()
df.reset_index(inplace=True, drop = True)
df

,Time,MW,temperature,CDH,omega1,omega2,omega3,omega4,omega5,omega6,...,omega159,omega160,omega161,omega162,omega163,omega164,omega165,omega166,omega167,omega168
0,2022-02-07 00:00:00,2.18,12.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-02-07 01:00:00,2.05,11.633781,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-02-07 03:00:00,1.71,12.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-02-07 05:00:00,1.7,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-02-07 14:00:00,2.5,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,2022-12-25 15:00:00,2.4,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3814,2022-12-25 17:00:00,2.6,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3815,2022-12-25 19:00:00,2.6,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3816,2022-12-25 21:00:00,2.6,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [5]:
'''
start_time = pd.Timestamp('2022-07-04 00:00:00')
end_time = pd.Timestamp('2022-08-28 23:00:00')
training_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time) & (df['CDH'] > 0)
df = df[training_mask]'''

"\nstart_time = pd.Timestamp('2022-07-04 00:00:00')\nend_time = pd.Timestamp('2022-08-28 23:00:00')\ntraining_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time) & (df['CDH'] > 0)\ndf = df[training_mask]"

Doing the temperature dependent part

In [6]:
start_time = pd.Timestamp('2022-08-01 00:00:00')
end_time = pd.Timestamp('2022-08-28 23:00:00')

#temperature dependency part
training_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time) & (df['CDH'] > 0)
training_df = df[training_mask]

d = training_df.loc[:,'MW']
phi = training_df.loc[:,'CDH']
opti = cd.Opti()
theta = opti.variable()
c = opti.variable()
sum = 0
residuals = []

for i in range (0,len(d)):
    residual = (d.iloc[i] - (phi.iloc[i] * theta + c))**2
    sum += residual
    residuals.append(residual)

#objective = sqrt(sum([r**2 for r in residuals]))
objective = sqrt(sum)
opti.minimize(objective)
opti.subject_to(theta >= 0)
# Solve the optimization problem
solver_opts = {'print_time': 0}
opti.solver('ipopt', solver_opts)
sol = opti.solve()

# Extract and print the optimal solution for theta
temp_theta = sol.value(theta)
print("Optimal value of theta:", temp_theta)
optimal_c = sol.value(c)
print("Optimal value of constant:", optimal_c)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        1
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

In [7]:
'''
training_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time)
behavior_df = df[training_mask]
e1 = 2.5
e2 = 2.5
#time dependent part
d = behavior_df['MW'] - temp_theta * behavior_df['CDH']
phi = behavior_df.loc[:,'omega1':'omega168']
opti = cd.Opti()
theta = opti.variable(168)
residuals = []
sum = 0
for i in range(len(d)):
    phi_i = (cd.MX(phi.iloc[i].values))
    phi_i =  cd.vertcat(phi_i)
    residual = (d.iloc[i] - cd.dot(phi_i, theta))**2
    sum += residual
    residuals.append(residual)

objective = sum
opti.minimize(objective)

for i in range(0, 167):
    opti.subject_to(cd.fabs(theta[i + 1] - theta[i]) <= e1)
    opti.subject_to((theta[i + 1] - theta[i]) <= e1)
    opti.subject_to((theta[i + 1] - theta[i]) >= -e1)
    opti.subject_to(theta[i] >= 0)
    opti.subject_to(theta[i] <= 7)

for i in range(1, 167):
    opti.subject_to((theta[i + 1] + theta[i - 1] - 2 * theta[i]) <= e2)
    opti.subject_to((theta[i + 1] + theta[i - 1] - 2 * theta[i]) >= -e2)

for i in range(7,8):
    month = bill_df.iloc[i]['Month'].month
    year = bill_df.iloc[i]['Month'].year
    num_days = calendar.monthrange(year, month)[1]
    first_day = calendar.monthrange(year, month)[0]
    num_hours = num_days * 24
    num_weeks = num_hours / 168.0
    remaining_hours = (num_days*24)%168
    week_sum = 0
    #for k in range(0,168):
        #week_sum+= theta[k]
    #sum = week_sum * num_complete_weeks
    #n = first_day * 24
    #for j in range(n, n+remaining_hours+1):
        #sum+=theta[j%168]
    weekly_energy_use = 0
    for j in range(0, 168):
        #sum+=theta[j]
        weekly_energy_use += theta[j] + temp_theta * behavior_df.iloc[j]['CDH']
    #opti.subject_to(sum == bill_df.iloc[i]['MW'])
    opti.subject_to(weekly_energy_use <= 1.5 * bill_df.iloc[i]['MW'] / num_weeks)

    opti.subject_to(weekly_energy_use >= 0.5 * bill_df.iloc[i]['MW'] / num_weeks)

# Solve the optimization problem
solver_opts = {'print_time': 0}
opti.solver('ipopt', solver_opts)
sol = opti.solve()
time_theta = sol.value(theta)
print("Optimal value of theta:", time_theta)
'''

'\ntraining_mask = (df[\'Time\'] >= start_time) & (df[\'Time\'] <= end_time)\nbehavior_df = df[training_mask]\ne1 = 2.5\ne2 = 2.5\n#time dependent part\nd = behavior_df[\'MW\'] - temp_theta * behavior_df[\'CDH\']\nphi = behavior_df.loc[:,\'omega1\':\'omega168\']\nopti = cd.Opti()\ntheta = opti.variable(168)\nresiduals = []\nsum = 0\nfor i in range(len(d)):\n    phi_i = (cd.MX(phi.iloc[i].values))\n    phi_i =  cd.vertcat(phi_i)\n    residual = (d.iloc[i] - cd.dot(phi_i, theta))**2\n    sum += residual\n    residuals.append(residual)\n\nobjective = sum\nopti.minimize(objective)\n\nfor i in range(0, 167):\n    opti.subject_to(cd.fabs(theta[i + 1] - theta[i]) <= e1)\n    opti.subject_to((theta[i + 1] - theta[i]) <= e1)\n    opti.subject_to((theta[i + 1] - theta[i]) >= -e1)\n    opti.subject_to(theta[i] >= 0)\n    opti.subject_to(theta[i] <= 7)\n\nfor i in range(1, 167):\n    opti.subject_to((theta[i + 1] + theta[i - 1] - 2 * theta[i]) <= e2)\n    opti.subject_to((theta[i + 1] + theta[i - 

In [8]:
'''
training_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time)
behavior_df = df[training_mask]
e1 = 2.0
e2 = 2.0
#time dependent part
d = behavior_df['MW'] - temp_theta * behavior_df['CDH']
phi = behavior_df.loc[:,'omega1':'omega168']
opti = cd.Opti()
theta = opti.variable(168)
residuals = []
sum = 0
for i in range(len(d)):
    phi_i = (cd.MX(phi.iloc[i].values))
    phi_i =  cd.vertcat(phi_i)
    residual = (d.iloc[i] - cd.dot(phi_i, theta))**2
    sum += residual
    residuals.append(residual)

objective = sum
opti.minimize(objective)

for i in range(0, 167):
    opti.subject_to(cd.fabs(theta[i + 1] - theta[i]) <= e1)
    opti.subject_to((theta[i + 1] - theta[i]) <= e1)
    opti.subject_to((theta[i + 1] - theta[i]) >= -e1)
    opti.subject_to(theta[i] >= 0)
    opti.subject_to(theta[i] <= 7)


for i in range(1, 167):
    opti.subject_to((theta[i + 1] + theta[i - 1] - 2 * theta[i]) <= e2)
    opti.subject_to((theta[i + 1] + theta[i - 1] - 2 * theta[i]) >= -e2)
#
this will be the code to apply the first constraint shared by sir
problem is that it is trying to eqaute sum of thetas to hte monthly power consumption of all the months
which will obviously not work

for i in range(0,13):
    month = bill_df.iloc[i]['Month'].month
    year = bill_df.iloc[i]['Month'].year
    num_days = calendar.monthrange(year, month)[1]
    first_day = calendar.monthrange(year, month)[0]
    num_hours = num_days * 24
    num_complete_weeks = num_hours // 168
    remaining_hours = (num_days*24)%168
    week_sum = 0
    for k in range(0,168):
        week_sum+= theta[k]
    sum = week_sum * num_complete_weeks
    n = first_day * 24
    for j in range(n, n+remaining_hours+1):
        sum+=theta[j%168]
    opti.subject_to(sum == bill_df.iloc[i]['MW'])
#
for i in range(7,8):
    month = bill_df.iloc[i]['Month'].month
    year = bill_df.iloc[i]['Month'].year
    num_days = calendar.monthrange(year, month)[1]
    first_day = calendar.monthrange(year, month)[0]
    num_hours = num_days * 24
    num_complete_weeks = num_hours // 168
    remaining_hours = (num_days*24)%168
    week_sum = 0
    for k in range(0,168):
        week_sum+= theta[k]
    sum = week_sum * num_complete_weeks
    n = first_day * 24
    for j in range(n, n+remaining_hours+1):
        sum+=theta[j%168]

    opti.subject_to(sum < bill_df.iloc[i]['MW'] + 0.01)

    opti.subject_to(sum > bill_df.iloc[i]['MW'] - 0.01)

# Solve the optimization problem
solver_opts = {'print_time': 0}
opti.solver('ipopt', solver_opts)
sol = opti.solve()
time_theta = sol.value(theta)
print("Optimal value of theta:", time_theta)
'''

"\nthis will be the code to apply the first constraint shared by sir\nproblem is that it is trying to eqaute sum of thetas to hte monthly power consumption of all the months\nwhich will obviously not work\n\nfor i in range(0,13):\n    month = bill_df.iloc[i]['Month'].month\n    year = bill_df.iloc[i]['Month'].year\n    num_days = calendar.monthrange(year, month)[1]\n    first_day = calendar.monthrange(year, month)[0]\n    num_hours = num_days * 24\n    num_complete_weeks = num_hours // 168\n    remaining_hours = (num_days*24)%168\n    week_sum = 0\n    for k in range(0,168):\n        week_sum+= theta[k]\n    sum = week_sum * num_complete_weeks\n    n = first_day * 24\n    for j in range(n, n+remaining_hours+1):\n        sum+=theta[j%168]\n    opti.subject_to(sum == bill_df.iloc[i]['MW'])\n"

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     2668
Number of nonzeros in Lagrangian Hessian.............:    14196

Total number of variables............................:      168
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:     1169
        inequality constraints with only lower bounds:      501
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      668

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.1816906e+04 3.61e+03 1.02e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [9]:
'''
#plotting the theta values
fig,ax = plt.subplots(figsize = (10,6))
ax.plot(np.arange(0,168), time_theta,'bo-')
'''


"\n#plotting the theta values\nfig,ax = plt.subplots(figsize = (10,6))\nax.plot(np.arange(0,168), time_theta,'bo-')\n"

In [10]:

#final model with df
training_df = df[training_mask]
d = training_df['MW']
c = np.concatenate((np.array([temp_theta]),time_theta),axis=0)
phi =training_df.loc[:,'CDH':'omega168']
full_model = LinearRegression(fit_intercept=False)
full_model.fit(phi, d)
full_model.coef_ = c
full_model.intercept_ = 0
yhat = full_model.predict(phi.values)
full_modelScore = full_model.score(phi,d)

print ('score for constructed full model on full data: ', full_modelScore)

fig,(ax2) = plt.subplots(nrows=1,figsize=(10,9))
_=ax2.plot(training_df['Time'],d,label='meas', marker = 'o')
_=ax2.plot(training_df['Time'],yhat,label='pred:', marker = 'x')
ax2.set_title('measured vs predicted data (full constructed model)')
_=ax2.legend()
plt.show()

LinearRegression(fit_intercept=False)

score for constructed full model on full data:  0.7783689550092832


D:\mlinternship\python\myenv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Text(0.5, 1.0, 'measured vs predicted data (full constructed model)')

In [11]:
#trying
import matplotlib.dates as mdates
#final model with DF
training_mask = (DF['Time'] >= start_time) & (DF['Time'] <= end_time)
training_df = DF[training_mask]
#actual_data = training_df['MW'].dropna()
training_df['MW'] = training_df['MW'].replace(np.nan, 0)
d = training_df['MW']
c = np.concatenate((np.array([temp_theta]),time_theta),axis=0)
phi =training_df.loc[:,'CDH':'omega168']
yhat = full_model.predict(phi.values)
actual_df = DF.copy()
actual_df = actual_df[training_mask]
actual_df.dropna(inplace = True)

fig10,(ax10) = plt.subplots(nrows=1,figsize=(10,9))
first_time = True
j = 0






for i in range(1, len(training_df)):
    if (j < len(actual_df)) and (training_df.iloc[i]['Time'] == actual_df.iloc[j]['Time']):
        j+=1
    else:
        iterator = i
        last_iterator = i-1
        first_time= False
        start =  mdates.date2num(training_df.iloc[last_iterator]['Time'])
        end =  mdates.date2num(training_df.iloc[iterator]['Time'])
        print(f'index: {i} start:{start}, width:{end-start}')
        rect = Rectangle((start, 0), end-start, 10, linewidth=1,  facecolor='green', alpha = 0.2, zorder = 10)
        ax10.add_patch(rect)


'''
for i in range(1, len(actual_df)):
    if  (actual_df.iloc[i]['Time'] == training_df.iloc[j]['Time']):
        j+=1
    else:
        iterator = i
        last_iterator = i-1
        first_time= False
        start =  mdates.date2num(actual_df.iloc[last_iterator]['Time'])
        end =  mdates.date2num(actual_df.iloc[iterator]['Time'])
        print(f'index: {i} start:{start}, width:{end-start}')
        rect = Rectangle((start, 0), end-start, 10, linewidth=1,  facecolor='green', alpha = 0.2, zorder = 10)
        ax10.add_patch(rect)
'''
locator = mdates.AutoDateLocator(minticks=3)
formatter = mdates.AutoDateFormatter(locator)
ax10.xaxis.set_major_locator(locator)
ax10.xaxis.set_major_formatter(formatter)
_=ax10.plot(actual_df['Time'][training_mask],actual_df[training_mask]['MW'],label='meas', marker = 'o', color = 'r', zorder = 5)
_=ax10.plot(training_df['Time'][training_mask],yhat,label='pred:', marker = 'x', color = 'y', zorder = 5)
ax10.set_title('measured vs predicted data (full constructed model with NA)')
_=ax10.legend()

plt.show()


index: 1 start:19205.0, width:0.041666666667879326


C:\Users\jaina\AppData\Local\Temp\ipykernel_33656\3995669895.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df['MW'] = training_df['MW'].replace(np.nan, 0)
D:\mlinternship\python\myenv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


index: 2 start:19205.041666666668, width:0.04166666666424135


index: 3 start:19205.083333333332, width:0.041666666667879326


index: 4 start:19205.125, width:0.041666666667879326


...

index: 668 start:19232.791666666668, width:0.04166666666424135


index: 671 start:19232.916666666668, width:0.04166666666424135


"\nfor i in range(1, len(actual_df)):\n    if  (actual_df.iloc[i]['Time'] == training_df.iloc[j]['Time']):\n        j+=1\n    else:\n        iterator = i\n        last_iterator = i-1\n        first_time= False\n        start =  mdates.date2num(actual_df.iloc[last_iterator]['Time'])\n        end =  mdates.date2num(actual_df.iloc[iterator]['Time'])\n        print(f'index: {i} start:{start}, width:{end-start}')\n        rect = Rectangle((start, 0), end-start, 10, linewidth=1,  facecolor='green', alpha = 0.2, zorder = 10)\n        ax10.add_patch(rect)\n"

C:\Users\jaina\AppData\Local\Temp\ipykernel_33656\3995669895.py:57: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  _=ax10.plot(actual_df['Time'][training_mask],actual_df[training_mask]['MW'],label='meas', marker = 'o', color = 'r', zorder = 5)


Text(0.5, 1.0, 'measured vs predicted data (full constructed model with NA)')

In [12]:
stop

NameError: name 'stop' is not defined

In [ ]:
#final model with DF
training_mask = (DF['Time'] >= start_time) & (DF['Time'] <= end_time)
training_df = DF[training_mask]
#actual_data = training_df['MW'].dropna()
training_df['MW'] = training_df['MW'].replace(np.nan, 0)
d = training_df['MW']
c = np.concatenate((np.array([temp_theta]),time_theta),axis=0)
phi =training_df.loc[:,'CDH':'omega168']
yhat = full_model.predict(phi.values)
actual_df = DF.copy()
actual_df = actual_df[training_mask]
actual_df.dropna(inplace = True)



fig10,(ax10) = plt.subplots(nrows=1,figsize=(10,9))
first_time = True
for i in range(0, len(actual_df)):
    if training_df.iloc[iterator+1]['Time'] == actual_df.iloc[i]['Time']:
        pass
    else:
        iterator = i
        while (training_df.iloc[iterator]['Time']!= actual_df.iloc[i]['Time']):
            iterator+=1
            if (iterator > len(actual_df)-1):
                iterator-=1
                break
        if first_time == True:
            last_iterator = i
            first_time= False

        start = training_df.iloc[last_iterator]['Time']
        width = training_df.iloc[iterator]['Time'] - training_df.iloc[last_iterator]['Time']
        rect = Rectangle((start, 0), width, 10, linewidth=1,  facecolor='green', alpha = 0.2, zorder = 10)
        ax10.add_patch(rect)
        last_iterator = iterator

_=ax10.plot(actual_df['Time'][training_mask],actual_df[training_mask]['MW'],label='meas', marker = 'o', color = 'r', zorder = 5)
_=ax10.plot(training_df['Time'][training_mask],yhat,label='pred:', marker = 'x', color = 'y', zorder = 5)
ax10.set_title('measured vs predicted data (full constructed model with NA)')
_=ax10.legend()

plt.show()



In [ ]:
stop

In [ ]:
#final model with DF
training_mask = (DF['Time'] >= start_time) & (DF['Time'] <= end_time)
training_df = DF[training_mask]
training_df['MW'] = training_df['MW'].replace(np.nan, 0)
phi = training_df.loc[:, 'CDH':'omega168']
yhat = full_model.predict(phi.values)
actual_df = DF.copy()
actual_df = actual_df[training_mask]
actual_df.dropna(inplace=True)

# Reset indices for both training_df and actual_df
training_df.reset_index(drop=True, inplace=True)
actual_df.reset_index(drop=True, inplace=True)
fig10, (ax10) = plt.subplots(nrows=1, figsize=(10, 9))

# Reset indices for consistency
training_df_reset = training_df.reset_index(drop=True)
actual_df_reset = actual_df.reset_index(drop=True)

# Initialize variables to track the start of mismatched regions
start_idx = 0

# Iterate through timestamps and plot rectangles for the mismatched regions
for i in range(len(actual_df_reset)):
    if training_df_reset['Time'][i] != actual_df_reset['Time'][i]:
        # End of mismatched region, plot a rectangle
        end_idx = i
        rect = Rectangle((start_idx, 0), end_idx - start_idx, 10, linewidth=1, facecolor='green', alpha=0.7, zorder=10)
        ax10.add_patch(rect)
        start_idx = i + 1  # Update start index for the next region

# Plot the actual and predicted data
_=ax10.plot(actual_df_reset['Time'], actual_df_reset['MW'], label='meas', marker='o', color='r', zorder=5)
_=ax10.plot(training_df_reset['Time'], yhat, label='pred:', marker='x', color='y', zorder=5)

ax10.set_title('measured vs predicted data (full constructed model with NA)')
_=ax10.legend()

plt.show()


In [ ]:
'''
PRCATICE
x = SX.sym("x",3)
f = x[0]**2 + x[1]**2 + x[2]**2
g1 = cd.fabs(x[2] - x[1]) - 1
g2 = cd.fabs(x[1] - x[0]) - 1
g3 = x[0] + x[1] + x[2] - 5
g4 = x[0]
g5 = x[1]
g6 = x[2]
g = cd.vertcat(g1,g2,g3,g4,g5,g6)
nlp = {'x': x, 'f': f, 'g': g}
MySolver = 'ipopt'
opts = {}
# Allocate a solver
solver = nlpsol("solver", MySolver, nlp, opts)
# Solve the NLP
lbg = [0,0,0, 0,0,0]
ubg = [inf, inf,5,inf,inf,inf]
sol = solver(lbg=lbg, ubg=ubg)
# Print solution
print("-----")
print("objective at solution = ", sol["f"])
print("primal solution = ", sol["x"])
print("dual solution (x) = ", sol["lam_x"])
print("dual solution (g) = ", sol["lam_g"])
'''
